In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv(r'/content/drive/MyDrive/train_10.csv', encoding='utf_8_sig')
test = pd.read_csv(r'/content/drive/MyDrive/test_10.csv', encoding='utf_8_sig')


In [4]:
train

,label,text,diff
0,negative,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
1,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
2,positive,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
3,negative,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...
4,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
5,negative,2021 license,diff --git a/Gruntfile.js b/Gruntfile.js\ninde...
6,negative,Merge pull request #2426 from alvarobartt/deve...,diff --git a/nltk/tag/stanford.py b/nltk/tag/s...
7,negative,Fixed link errors,diff --git a/src/irisnet/CMakeLists.txt b/src/...
8,positive,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...
9,negative,Bootstrap 3 migration: alert-error->alert-danger.,diff --git a/src/main/java/org/projectforge/we...


In [5]:
len(train)

20

In [6]:
train['label'].value_counts()

negative    13
positive     7
Name: label, dtype: int64

In [7]:
test['label'].value_counts()

negative    6334
positive    3758
Name: label, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,negative,Header: clarify that newline conversion is raw...,diff --git a/lib/mail/header.rb b/lib/mail/hea...,"[-0.13944659, 0.07307362, 0.040002007, -0.0487..."
1,negative,[ticket/16691] Fix illegal offset for report_p...,diff --git a/phpBB/phpbb/notification/type/rep...,"[-0.14759508, 0.0932299, 0.033147737, 0.060865..."
2,negative,Hide the Ulterius Agent when its ran.,diff --git a/UlteriusAgent/Program.cs b/Ulteri...,"[0.0017121456, 0.030022606, -0.0066839824, 0.2..."
3,negative,Move DOM lookup outside of loop\n\nfixes #1360,diff --git a/lib/ext/ui.js b/lib/ext/ui.js\nin...,"[-0.04255979, 0.13228248, -0.090833075, 0.1190..."
4,positive,remove /login from tracking logs so we don't c...,diff --git a/auth/views.py b/auth/views.py\nin...,"[-0.052938215, 0.12112449, 0.07753473, 0.19624..."
...,...,...,...,...
10087,positive,disallow boot from volume from specifying arbi...,diff --git a/nova/compute/api.py b/nova/comput...,"[-0.041516613, -0.017604226, 0.0018733417, 0.0..."
10088,positive,writet1 protection against buffer overflow\n\n...,diff --git a/texk/dvipsk/ChangeLog b/texk/dvip...,"[-0.15439484, 0.04666136, 0.04632255, 0.121904..."
10089,positive,PR/398: Correctly truncate pascal strings (fix...,diff --git a/src/softmagic.c b/src/softmagic.c...,"[0.026086219, -0.08980047, -0.02994169, 0.1276..."
10090,negative,Add paragraph on GitHub deprecating passwords ...,diff --git a/CONTRIBUTING-TO-ESAPI.txt b/CONTR...,"[0.10408172, 0.0822196, 0.029082187, 0.0035575..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.7128418549346016
Logistic Regression Precision: 0.707302204880861
Logistic Regression Recall: 0.7128418549346016
Logistic Regression Recall: 0.6943032910561456
